tree census data exploration.  starting with 2015 census

In [2]:
import pandas as pd 
import matplotlib


In [3]:
tree_df = pd.read_csv("new_york_tree_census_2015.csv")
tree_df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,2016-06-28,10,0,OnCurb,Alive,Good,Fraxinus pennsylvanica,green ash,...,25,14,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,2015-08-19,9,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,34,13,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,2015-12-30,7,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,32,10,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,2016-07-05,10,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,46,22,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,2015-06-13,4,0,OnCurb,Alive,Good,Prunus virginiana,'Schubert' chokecherry,...,31,10,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017


In [4]:
remove_values = ["Dead", "Stump"]
alive_df = tree_df[~tree_df["status"].isin(remove_values)]
alive_df.describe()

,tree_id,block_id,tree_dbh,stump_diam,zipcode,cb_num,borocode,cncldist,st_assem,st_senate,boro_ct,latitude,longitude,x_sp,y_sp
count,652173.000000,652173.000000,652173.000000,652173.0,652173.000000,652173.000000,652173.000000,652173.000000,652173.000000,652173.000000,6.521730e+05,652173.000000,652173.000000,6.521730e+05,652173.000000
mean,365689.637480,313611.213077,11.709485,0.0,10914.407958,343.738388,3.360979,29.992976,50.818103,20.620725,3.407340e+06,40.700991,-73.924587,1.005133e+06,194699.696220
std,208194.926983,114775.077889,8.634239,0.0,656.868397,115.808325,1.167572,14.343667,18.928937,7.378427,1.176612e+06,0.090268,0.123844,3.435720e+04,32886.140551
min,3.000000,100002.000000,0.000000,0.0,83.000000,101.000000,1.000000,1.000000,23.000000,10.000000,1.000201e+06,40.498466,-74.254965,9.133493e+05,120973.792200
25%,186886.000000,221491.000000,5.000000,0.0,10314.000000,303.000000,3.000000,19.000000,33.000000,14.000000,3.012000e+06,40.631725,-73.980973,9.895259e+05,169441.751400
50%,367204.000000,319886.000000,10.000000,0.0,11214.000000,402.000000,4.000000,30.000000,52.000000,21.000000,4.008100e+06,40.700287,-73.913449,1.008235e+06,194444.837500
75%,546632.000000,404665.000000,16.000000,0.0,11365.000000,412.000000,4.000000,43.000000,64.000000,25.000000,4.103300e+06,40.761999,-73.835219,1.029902e+06,216940.389000
max,722694.000000,999999.000000,425.000000,0.0,11697.000000,503.000000,5.000000,51.000000,87.000000,36.000000,5.032300e+06,40.912918,-73.700488,1.067248e+06,271894.092100


In [5]:
alive_df.columns

Index(['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam',
       'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward',
       'guards', 'sidewalk', 'user_type', 'problems', 'root_stone',
       'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
       'brch_light', 'brch_shoe', 'brch_other', 'address', 'zipcode',
       'zip_city', 'cb_num', 'borocode', 'boroname', 'cncldist', 'st_assem',
       'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude',
       'longitude', 'x_sp', 'y_sp'],
      dtype='object')

In [6]:
filtered_df = alive_df.drop(['created_at', 'stump_diam', 'status', 'user_type', 'sidewalk', 'guards', 'steward', 'st_assem', 
                             'st_senate', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
                             'brch_light', 'brch_shoe', 'brch_other'], axis=1)
filtered_df.head()

,tree_id,block_id,tree_dbh,curb_loc,health,spc_latin,spc_common,problems,address,zipcode,...,boroname,cncldist,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,10,OnCurb,Good,Fraxinus pennsylvanica,green ash,Stones,76-046 164 STREET,11366,...,Queens,24,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,9,OnCurb,Good,Gleditsia triacanthos var. inermis,honeylocust,BranchLights,72-020 32 AVENUE,11370,...,Queens,25,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,7,OnCurb,Good,Pyrus calleryana,Callery pear,BranchLights,153-026 119 AVENUE,11434,...,Queens,28,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,10,OnCurb,Good,Pyrus calleryana,Callery pear,NaN,89 89 STREET,11209,...,Brooklyn,43,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,4,OnCurb,Good,Prunus virginiana,'Schubert' chokecherry,BranchLights,559 BEACH 68 STREET,11692,...,Queens,31,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017
